# Writing questions about Clinvar

Goal: Evaluate RAG

## Data preparation

In [130]:
import pandas as pd
import json
from pprint import pprint
import polars as pl
import random


df = pl.read_csv("../../data/mitocarta/Human.MitoCarta3.0.csv", infer_schema_length=None)

pl.Config.set_tbl_cols(-1)
print(df.head())
print(df.columns)
print(df.describe())

shape: (5, 48)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐
│ Hum ┆ Mou ┆ Sym ┆ Syn ┆ Des ┆ Mit ┆ Mit ┆ Mit ┆ Mit ┆ Tra ┆ Mit ┆ Mit ┆ Mit ┆ Mit ┆ Ens ┆ Uni ┆ Pro ┆ Tar ┆ Mit ┆ Coe ┆ PGC ┆ Yea ┆ Ric ┆ MSM ┆ hg1 ┆ hg1 ┆ hg1 ┆ MSM ┆ MSM ┆ MSM ┆ MSM ┆ MSM ┆ Tis ┆ cer ┆ cer ┆ bra ┆ spi ┆ kid ┆ liv ┆ hea ┆ ske ┆ adi ┆ sma ┆ lar ┆ sto ┆ pla ┆ tes ┆ HPA │
│ anG ┆ seO ┆ bol ┆ ony ┆ cri ┆ oCa ┆ oCa ┆ oCa ┆ oCa ┆ ini ┆ oCa ┆ oCa ┆ oCa ┆ oCa ┆ emb ┆ Pro ┆ tei ┆ get ┆ oDo ┆ xpr ┆ _In ┆ stM ┆ ket ┆ S_S ┆ 9_C ┆ 9_S ┆ 9_S ┆ S_N ┆ S_N ┆ S_N ┆ S_T ┆ S_P ┆ sue ┆ ebr ┆ ebe ┆ ins ┆ nal ┆ ney ┆ er_ ┆ rt_ ┆ let ┆ pos ┆ lli ┆ gei ┆ mac ┆ cen ┆ tis ┆ _Ma │
│ ene ┆ rth ┆ --- ┆ ms  ┆ pti ┆ rta ┆ rta ┆ rta ┆ rta ┆ ngD ┆ rta ┆ rta ┆ rta ┆ rta ┆ lGe ┆ t   ┆ nLe ┆ P_S ┆ mai ┆

##### Affichage des colonnes (valeurs fréquentes)

In [ ]:
for col in df:
    print(col.value_counts().sort(by="count", descending=True))

shape: (1_137, 2)
┌─────────────┬───────┐
│ HumanGeneID ┆ count │
│ ---         ┆ ---   │
│ str         ┆ u32   │
╞═════════════╪═══════╡
│ null        ┆ 21    │
│ 6341        ┆ 1     │
│ 80224       ┆ 1     │
│ 65991       ┆ 1     │
│ 55621       ┆ 1     │
│ …           ┆ …     │
│ 84263       ┆ 1     │
│ 192286      ┆ 1     │
│ 254552      ┆ 1     │
│ 80222       ┆ 1     │
│ 64756       ┆ 1     │
└─────────────┴───────┘
shape: (1_117, 2)
┌─────────────────────┬───────┐
│ MouseOrthologGeneID ┆ count │
│ ---                 ┆ ---   │
│ i64                 ┆ u32   │
╞═════════════════════╪═══════╡
│ null                ┆ 41    │
│ 67212               ┆ 1     │
│ 56284               ┆ 1     │
│ 76722               ┆ 1     │
│ 14775               ┆ 1     │
│ …                   ┆ …     │
│ 56046               ┆ 1     │
│ 56327               ┆ 1     │
│ 17991               ┆ 1     │
│ 215708              ┆ 1     │
│ 84682               ┆ 1     │
└─────────────────────┴───────┘
shape: (1_13

### Quel est le symbole du gène humain correspondant à l’identifiant Ensembl {EnsemblGeneID} ?

In [133]:
n = 5

# =============== DATA PREPARATION ===============
# 1. Extract the gene symbol column, removes nulls, duplications
ensembl_id_in_mitocarta3 = (
    df
    .select(pl.col("EnsemblGeneID_mapping_version_20200130"))
    .drop_nulls()
    .unique()
    .to_series()
    .to_list()
)
random_ensembl_id = random.sample(ensembl_id_in_mitocarta3, n)

# =============== ENSEMBL ID -> GENE  ===============

results = []

for ensembl in random_ensembl_id:
    genes = (
        df
        .filter(pl.col("EnsemblGeneID_mapping_version_20200130").str.contains(ensembl))
        .select("Symbol")
        .unique()
    )

    gene = genes["Symbol"][0]
    results.append({"EnsemblGeneID": ensembl, "Gene": gene})

df_results = pl.DataFrame(results)


# =============== PRINT  ===============

for row in df_results.iter_rows():
    EnsemblGeneID = row[0]
    Gene = row[1]

    print(f"Question: Quel est le symbole du gène humain correspondant à l’identifiant Ensembl {EnsemblGeneID} ?")
    print(f"Answer: Le symbole de l'identifiant Ensembl {EnsemblGeneID} est {Gene}.")
    print()


Question: Quel est le symbole du gène humain correspondant à l’identifiant Ensembl ENSG00000134375 ?
Answer: Le symbole de l'identifiant Ensembl ENSG00000134375 est TIMM17A.

Question: Quel est le symbole du gène humain correspondant à l’identifiant Ensembl ENSG00000130348 ?
Answer: Le symbole de l'identifiant Ensembl ENSG00000130348 est QRSL1.

Question: Quel est le symbole du gène humain correspondant à l’identifiant Ensembl ENSG00000082515 ?
Answer: Le symbole de l'identifiant Ensembl ENSG00000082515 est MRPL22.

Question: Quel est le symbole du gène humain correspondant à l’identifiant Ensembl ENSG00000110717 ?
Answer: Le symbole de l'identifiant Ensembl ENSG00000110717 est NDUFS8.

Question: Quel est le symbole du gène humain correspondant à l’identifiant Ensembl ENSG00000164366 ?
Answer: Le symbole de l'identifiant Ensembl ENSG00000164366 est CCDC127.



### Quelle est la localisation sub-mitochondriale du gène {Symbol} ?

- "MitoCarta3.0_SubMitoLocalization" = MitoCarta (Broad Institute, 2020–2021); Liste spécifique de protéines mitochondriales
- "HPA_Main_Location_2020 (Reliability)" = Human Protein Atlas (HPA, 2020); Localisation cellulaire générale (mitochondrie + autres organites)

In [47]:
(
    df
    .select("MitoCarta3.0_SubMitoLocalization")
    .drop_nulls()
    .unique()
    .to_series()
    .to_list()
)

['Matrix',
 'MIM',
 'Membrane',
 'MOM|IMS',
 'MOM|Matrix',
 'MOM',
 'IMS',
 'Matrix|IMS',
 'unknown']

In [ ]:
n = 5

# =============== DATA PREPARATION ===============
gene_in_mitocarta3 = (
    df
    .select(pl.col("Symbol"))
    .drop_nulls()
    .unique()
    .to_series()
    .to_list()
)

random_gene = random.sample(gene_in_mitocarta3, n)

# =============== ENSEMBL ID -> GENE  ===============

results = []

for gene in random_gene:
    location = (
        df
        .filter(pl.col("Symbol") == gene)
        .select("MitoCarta3.0_SubMitoLocalization")
        .unique()
        .to_series()
        .to_list()
    ) # No gestion of null values.
    results.append({"Gene": gene, "Location": location[0]})

df_results = pl.DataFrame(results)
print(df_results)

# =============== PRINT  ===============
# matrix, MIM (mitochondrial inner membrane), IMS (intermembrane space), MOM (mitochondrial outer membrane), membrane, unknown. 

for row in df_results.iter_rows():
    Gene = row[0]
    Location = row[1].split("|")

    print(f"Question:  Quelle est la localisation sub-mitochondriale du gène {Gene} ?")
    print(f"Answer: Le gèhne {Gene} est localisé dans {", ".join(loc for loc in Location)}.")
    print()

## Amélioration : prise en compte des absences. Prise en compte des abbréviations.


shape: (5, 2)
┌────────┬──────────┐
│ Gene   ┆ Location │
│ ---    ┆ ---      │
│ str    ┆ str      │
╞════════╪══════════╡
│ PDHA2  ┆ Matrix   │
│ NDUFS2 ┆ MIM      │
│ ACOT7  ┆ Matrix   │
│ APEX1  ┆ Matrix   │
│ CPOX   ┆ MIM      │
└────────┴──────────┘
Question:  Quelle est la localisation sub-mitochondriale du gène PDHA2 ?
Answer: Le gèhne PDHA2 est localisé dans Matrix.

Question:  Quelle est la localisation sub-mitochondriale du gène NDUFS2 ?
Answer: Le gèhne NDUFS2 est localisé dans MIM.

Question:  Quelle est la localisation sub-mitochondriale du gène ACOT7 ?
Answer: Le gèhne ACOT7 est localisé dans Matrix.

Question:  Quelle est la localisation sub-mitochondriale du gène APEX1 ?
Answer: Le gèhne APEX1 est localisé dans Matrix.

Question:  Quelle est la localisation sub-mitochondriale du gène CPOX ?
Answer: Le gèhne CPOX est localisé dans MIM.



### Quelles preuves expérimentales soutiennent l’inclusion du gène {Symbol} dans MitoCarta ?

In [52]:
print(df["MitoCarta3.0_Evidence"].unique())

shape: (820,)
Series: 'MitoCarta3.0_Evidence' [str]
[
	"targetP signal, yeast mito homolog, mito protein domain+, coexpression+"
	"literature, APEX_IMS, APEX_matrix, Rickettsial homolog, mito protein domain, coexpression, MS/MS++"
	"Rickettsial homolog, mito protein domain+"
	"literature, mito protein domain+, MS/MS"
	"literature, APEX_matrix, targetP signal, yeast mito homolog+, Rickettsial homolog, mito protein doma…
	…
	"APEX_matrix, yeast mito homolog++, Rickettsial homolog, coexpression, MS/MS+"
	"literature, yeast mito homolog+, Rickettsial homolog, mito protein domain, coexpression, MS/MS"
	"targetP signal, mito protein domain, coexpression, MS/MS"
	"literature, APEX_matrix, GFP, Rickettsial homolog, mito protein domain, coexpression, MS/MS++"
	"literature, mito protein domain+, induction, coexpression, MS/MS"
]


In [56]:
n = 5

# =============== DATA PREPARATION ===============
gene_in_mitocarta3 = (
    df
    .select(pl.col("Symbol"))
    .drop_nulls()
    .unique()
    .to_series()
    .to_list()
)

random_gene = random.sample(gene_in_mitocarta3, n)

# =============== GENE -> EVIDENCE  ===============

results = []

for gene in random_gene:
    evidence = (
        df
        .filter(pl.col("Symbol") == gene)
        .select("MitoCarta3.0_Evidence")
        .unique()
        .to_series()
        .to_list()
    ) # No gestion of null values.
    results.append({"Gene": gene, "Evidence": evidence[0]})

df_results = pl.DataFrame(results)
print(df_results)

# =============== PRINT  ===============
# matrix, MIM (mitochondrial inner membrane), IMS (intermembrane space), MOM (mitochondrial outer membrane), membrane, unknown. 

for row in df_results.iter_rows():
    Gene = row[0]
    Evidence = row[1].split(",")

    print(f"Question:  Quelles preuves expérimentales soutiennent l’inclusion du gène {Gene} dans MitoCarta ?")
    print(f"Answer: L'inclusion du gène {Gene} dans Mitocarta 3 est prouvé par {", ".join(ev for ev in Evidence)}.")
    print()

## Amélioration : prise en compte des absences. Prise en compte des abbréviations. Prise en compte de la version.

shape: (5, 2)
┌───────┬───────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Gene  ┆ Evidence                                                                                              │
│ ---   ┆ ---                                                                                                   │
│ str   ┆ str                                                                                                   │
╞═══════╪═══════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ MCCC1 ┆ literature, targetP signal, Rickettsial homolog, mito protein domain, induction, coexpression+, MS/M… │
│ DCAKD ┆ GFP, yeast mito homolog, Rickettsial homolog, mito protein domain, coexpression, MS/MS                │
│ GRSF1 ┆ APEX_matrix, mito protein domain, induction, coexpression, MS/MS                                      │
│ MTFP1 ┆ literature, induction, coexpression, MS/MS+                     

### Quelle est la longueur de la protéine codée par le gène {Symbol} ?

In [ ]:
print(df["ProteinLength"])

shape: (1_157,)
Series: 'ProteinLength' [i64]
[
	325
	280
	217
	664
	480
	…
	null
	null
	null
	null
	null
]


In [68]:
n = 5

# =============== DATA PREPARATION ===============
gene_in_mitocarta3 = (
    df
    .select(pl.col("Symbol"))
    .drop_nulls()
    .unique()
    .to_series()
    .to_list()
)

random_gene = random.sample(gene_in_mitocarta3, n)

# =============== GENE -> PROTEINLENGTH  ===============

results = []

for gene in random_gene:
    ProteinLength = (
        df
        .filter(pl.col("Symbol") == gene)
        .select("ProteinLength")
        .unique()
        .to_series()
        .to_list()
    ) # No gestion of null values.
    results.append({"Gene": gene, "ProteinLength": ProteinLength[0]})

df_results = pl.DataFrame(results)
print(df_results)

# =============== PRINT  ===============
# matrix, MIM (mitochondrial inner membrane), IMS (intermembrane space), MOM (mitochondrial outer membrane), membrane, unknown. 

for row in df_results.iter_rows():
    Gene = row[0]
    ProteinLength = row[1]

    print(f"Question:  Quelle est la longueur de la protéine codée par le gène {Gene} ?")
    print(f"Answer: Le gène {Gene} donne une protéine de longueur : {ProteinLength} acides aminés.")
    print()

## Amélioration : prise en compte des absences. Prise en compte des abbréviations. Prise en compte de la version.

shape: (5, 2)
┌────────┬───────────────┐
│ Gene   ┆ ProteinLength │
│ ---    ┆ ---           │
│ str    ┆ i64           │
╞════════╪═══════════════╡
│ CYC1   ┆ 325           │
│ COA7   ┆ 231           │
│ HADHB  ┆ 474           │
│ MRPL38 ┆ 380           │
│ D2HGDH ┆ 521           │
└────────┴───────────────┘
Question:  Quelle est la longueur de la protéine codée par le gène CYC1 ?
Answer: Le gène CYC1 donne une protéine de longueur : 325 acides aminés.

Question:  Quelle est la longueur de la protéine codée par le gène COA7 ?
Answer: Le gène COA7 donne une protéine de longueur : 231 acides aminés.

Question:  Quelle est la longueur de la protéine codée par le gène HADHB ?
Answer: Le gène HADHB donne une protéine de longueur : 474 acides aminés.

Question:  Quelle est la longueur de la protéine codée par le gène MRPL38 ?
Answer: Le gène MRPL38 donne une protéine de longueur : 380 acides aminés.

Question:  Quelle est la longueur de la protéine codée par le gène D2HGDH ?
Answer: Le gène

### Dans combien de tissus le gène {Symbol} a-t-il été détecté par MS/MS ?

Normally TISSUES column is related to MSMS_NUM_TISSUES. Only did a visual check.

In [71]:
# for row in df.select("Symbol", "Tissues", "MSMS_NUM_TISSUES").iter_rows():
#     print(row)

In [ ]:
n = 5

# =============== DATA PREPARATION ===============
gene_in_mitocarta3 = (
    df
    .select(pl.col("Symbol"))
    .drop_nulls()
    .unique()
    .to_series()
    .to_list()
)

random_gene = random.sample(gene_in_mitocarta3, n)

# =============== GENE -> TISSUES  ===============

results = []

for gene in random_gene:
    Tissues = (
        df
        .filter(pl.col("Symbol") == gene)
        .select("Tissues")
        .unique()
        .to_series()
        .to_list()
    ) # No gestion of null values.
    results.append({"Gene": gene, "Tissues": Tissues[0]})

df_results = pl.DataFrame(results)
print(df_results)

# =============== PRINT  ===============
# matrix, MIM (mitochondrial inner membrane), IMS (intermembrane space), MOM (mitochondrial outer membrane), membrane, unknown. 

for row in df_results.iter_rows():
    Gene = row[0]
    Tissues = row[1].split(",")

    print(f"Question: Dans combien de tissus le gène {Gene} a-t-il été détecté par MS/MS ?")
    print(f"Answer: Le gène {Gene} a été détecté dans les {len(Tissues) if Tissues else 0} tissus suivants : {", ".join(tissue for tissue in Tissues)}.")
    print()

## Amélioration : prise en compte des absences. Prise en compte des abbréviations. Prise en compte de la version.

shape: (5, 2)
┌────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Gene   ┆ Tissues                                                                                               │
│ ---    ┆ ---                                                                                                   │
│ str    ┆ str                                                                                                   │
╞════════╪═══════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ NIT1   ┆ cerebrum, cerebellum, brainstem, spinalcord, kidney, liver, adipose, smallintestine, largeintestine,… │
│ IMMP2L ┆ cerebellum, brainstem, liver, skeletalmuscle, adipose, largeintestine, testis                         │
│ COX11  ┆ cerebrum, cerebellum, spinalcord, skeletalmuscle, largeintestine, placenta                            │
│ MRPL43 ┆ all 14                                                 

### Quels gènes sont localisés dans la {SubMitoLocalization} selon Mitocarta3.0 ? 

In [92]:
n = 1

# =============== MITOCHONDRIAL MATRIX -> GENES ===============

mito_sublocalization = (
    df
    .select("MitoCarta3.0_SubMitoLocalization")
    .drop_nulls()
    .unique()
    .to_series()
    .to_list()
)

unique_mito_subloc = set() # Transfo to avoid combinations like "MIM|Matrix" in random selection.
for el in mito_sublocalization:
    unique_mito_subloc.update(el.split("|"))

random_sub_mito_localization = random.sample(list(unique_mito_subloc), n)


gene_in_subMitoLocalization_in_mitocarta3 = (
    df
    .filter(pl.col("MitoCarta3.0_SubMitoLocalization").str.contains(random_sub_mito_localization[0]))
    .select("Symbol")
    .drop_nulls()
    .unique()
    .to_series()
    .to_list()
)
# print(gene_in_subMitoLocalization_in_mitocarta3)

# ============== PRINT =====================

print(f"Question: Quels gènes sont localisés dans la {random_sub_mito_localization[0]} selon Mitocarta3.0 ? ")
print(f"Answer: On retrouve dans le {random_sub_mito_localization[0]}, les gènes suivants : {", ".join(gene for gene in gene_in_subMitoLocalization_in_mitocarta3)}.")
print()

## Amélioration : 
# - prise en compte des absences. 
# - Prise en compte des abbréviations. 
# - Prise en compte de la version.
# - Prise en compte de plusieurs critères comme deux sous-localisations simultanées

Question: Quels gènes sont localisés dans la Membrane selon Mitocarta3.0 ? 
Answer: On retrouve dans le Membrane, les gènes suivants : AIFM2, GPAM, ARMCX6, MGST1, IFI27, ABCD3, CCDC127, ABCA9, NAT8L, SPTLC2, MTFR2, DCAKD, PRXL2A, DNAJC4, ALDH3A2, MGARP, STOM, C15orf48, ABCD2, ABCD1, DHRS7B, COMT, TMEM205, ARMCX2, HTATIP2, PLGRKT, ADCY10, MTFP1, LETMD1, PEX11B, MTFR1, COMTD1, TDRKH, RDH14.



### Quels gènes appartiennent à la voie {Pathway} ?

In [134]:

# =============== PATHWAY -> GENES ===============
n = 1
all_pathways = (
    df
    .select("MitoCarta3.0_MitoPathways")
    .unique()
    .drop_nulls()
    .to_series()
    .to_list()
)

all_pathways_sorted = set()

for path in all_pathways:
    modified_path = path.replace(">", ";").replace("|", ";")
    all_pathways_sorted.update(el.strip() for el in modified_path.split(";"))
    
random_pathway = random.sample(list(all_pathways_sorted), n)


gene_in_mito_pathways_in_mitocarta3 = (
    df
    .filter(pl.col("MitoCarta3.0_MitoPathways").str.contains(random_pathway[0]))
    .select("Symbol")
    .drop_nulls()
    .unique()
    .to_series()
    .to_list()
)

print(random_pathway, gene_in_mito_pathways_in_mitocarta3)

# ============== PRINT =====================

print(f"Question: Quels gènes appartiennent à la voie métabolique {random_pathway[0]} ? ")
print(f"Answer: On retrouve dans la voie métabolique {random_pathway[0]}, les gènes suivants : {", ".join(gene for gene in gene_in_mito_pathways_in_mitocarta3)}.")
print()

['Fusion'] ['MIGA1', 'ARL2', 'MTCH2', 'PLD6', 'MIGA2', 'OMA1', 'OPA1', 'MFN1', 'MFN2']
Question: Quels gènes appartiennent à la voie métabolique Fusion ? 
Answer: On retrouve dans la voie métabolique Fusion, les gènes suivants : MIGA1, ARL2, MTCH2, PLD6, MIGA2, OMA1, OPA1, MFN1, MFN2.



### Quels gènes mitochondriaux sont exprimés dans tous les 14 tissus analysés ? (variante Plus ou moins de tissus)

In [ ]:
n = 1

# =============== 14 TISSUES -> GENES  ===============

genes_expressed_in_every_tissues = (
    df
    .filter(pl.col("MSMS_NUM_TISSUES") == 14)
    .select("Symbol")
    .unique()
    .to_series()
    .to_list()
) # Need to change df["MSMS_NUM_TISSUES"] column type.

print(genes_expressed_in_every_tissues)

# =============== PRINT  ===============
# matrix, MIM (mitochondrial inner membrane), IMS (intermembrane space), MOM (mitochondrial outer membrane), membrane, unknown. 

# for row in df_results.iter_rows():
#     Gene = row[0]
#     Tissues = row[1].split(",")

#     print(f"Question: Dans combien de tissus le gène {Gene} a-t-il été détecté par MS/MS ?")
#     print(f"Answer: Le gène {Gene} a été détecté dans les {len(Tissues) if Tissues else 0} tissus suivants : {", ".join(tissue for tissue in Tissues)}.")
#     print()

## Amélioration : prise en compte des absences. Prise en compte des abbréviations. Prise en compte de la version.

### Question templates

- [x] Quel est le symbole du gène humain correspondant à l’identifiant Ensembl {EnsemblGeneID} ?
- [x] Quelle est la localisation sub-mitochondriale du gène {Symbol} ?
- [x] Quelles preuves expérimentales soutiennent l’inclusion du gène {Symbol} dans MitoCarta ? // Est-ce que la protéine {Symbol} est présente dans la mitochondrie, si oui selon quelles preuves ?
- [x] Quelle est la longueur de la protéine codée par le gène {Symbol} ?
- [x] Dans combien de tissus le gène {Symbol} a-t-il été détecté par MS/MS ?
- [x] Quels gènes sont localisés dans la {SubMitoLocalization} selon Mitocarta3.0 ? 
- [x] Quels gènes appartiennent à la voie OXPHOS > Complex I ?
- [ ] Quels gènes mitochondriaux sont exprimés dans tous les 14 tissus analysés ?
- [ ] Quelle différence de localisation sub-mitochondriale existe entre les gènes {GeneA} et {GeneB} ?
- [ ] Entre {GeneA} et {GeneB}, lequel présente la plus forte intensité protéique dans le foie ?
- [ ] Le gène {Symbol} a-t-il une homologation mitochondriale plus forte chez la souris que dans la levure ?
- [ ] Comment le score MS/MS du gène {Symbol} se compare-t-il à la moyenne des gènes de la même voie ?
- [ ] Quels gènes présentent à la fois une localisation mitochondriale confirmée (HPA) et un score MS/MS > 50 % ?
- [ ] Quels gènes montrent une expression préférentielle dans le cerveau avec une fonction de traduction mitochondriale ?
- [ ] Interprétez le TargetP_Score = 5 du gène {Symbol} : que suggère-t-il sur le signal d’adressage mitochondrial ?
- [ ] Comment comprendre un MitoDomain_Score = NonMitoDomain pour un gène pourtant inclus dans MitoCarta ? (Intéressante)
- [ ] Quelle signification clinique pourrait avoir un FDR proche de 0 dans MitoCarta ?
- [ ] Parmi les gènes mitochondriaux exprimés dans le cerveau, lesquels sont liés à des pathologies neurodégénératives ?
- [ ] Quels gènes mitochondriaux fortement exprimés dans le cœur sont candidats à une cardiomyopathie mitochondriale ?
- [ ] Quels gènes de MitoCarta3.0 sont impliqués dans le métabolisme lipidique mitochondrial ?
- [ ] Existe-t-il des gènes mitochondriaux sans orthologue murin ? Quelle importance pour les modèles animaux ?
- [ ] Quels gènes présentent une localisation sub-mitochondriale indéterminée ou inconnue ?
- [ ] Pour combien de gènes les preuves expérimentales sont uniquement issues de la littérature ?
- [ ] Quels gènes de MitoCarta3.0 présentent des scores contradictoires entre différentes méthodes d’évidence ?
- [ ] Quels sont les gènes présents à la fois dans la sous-localisation mitochondriale {sub1} et {sub2} ? 
- [ ] Quelle voie métabolique implique le plus de gènes ?

Revoir:
- Listez les gènes ayant une homologation mitochondriale confirmée dans la levure.
- Comment la présence d’un domaines protéiques mitochondriaux influence-t-elle la probabilité d’appartenance à MitoCarta ?
- Quels gènes associés à la fission mitochondriale pourraient impacter la survie neuronale ?
- Quelle corrélation existe entre l’intensité dans les tissus cardiaques et le rôle dans la chaîne respiratoire ?
- Quels gènes présentent un score de coexpression élevé (>30) ?

Pour aller + LOIN :

- Est-ce qu'on peut théoriser le code pour qu'il soit universel selon la question posée et selon le tableau ?
- L'utilisation d'un gène plus ancien qui est synonyme ? 
- La version demandée ou affichée
- Les publications associées